In [1]:
! pip install -q transformers
! cp drive/My\ Drive/Colab\ Notebooks/*.csv ./

     |████████████████████████████████| 890kB 2.9MB/s 
     |████████████████████████████████| 890kB 19.2MB/s 
     |████████████████████████████████| 1.1MB 17.7MB/s 
     |████████████████████████████████| 3.0MB 27.0MB/s 


In [2]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('dankmemes_task1_train.csv')
embedding = pd.read_csv('dankmemes_task1_train_embeddings.csv', header=None)

In [4]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-multilingual-cased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [6]:
tokenized = df['Text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [7]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded_all = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [8]:
features = []

for i in range(16):
  padded = padded_all[i:i+100]
  input_ids = torch.tensor(padded)
  attention_mask = torch.tensor(np.where(padded != 0, 1, 0))
  with torch.no_grad():
      features.extend(model(input_ids, attention_mask=attention_mask)[0][:,0,:].numpy().tolist())

KeyboardInterrupt: ignored

In [ ]:
features = np.array(features)

In [ ]:
import pickle
import os

pickle.dump(features,
            open(os.path.join('bert_embeddings.pkl'), 'wb'),
            protocol=4)

In [ ]:
! mv bert_embeddings.pkl ./drive/My\ Drive/Colab\ Notebooks/